In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2021-10-22 14:47:58--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.253.119.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  89.1MB/s    in 1.6s    

2021-10-22 14:48:00 (89.1 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2021-10-22 14:48:00--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.128, 64.233.191.128, 173.194.74.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (

In [3]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    fill_mode = 'nearest',
    zoom_range=0.2,
    horizontal_flip=True,
)

In [5]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

In [6]:
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size = 64,
    target_size = (176, 176),
    class_mode = 'binary',
)

Found 1027 images belonging to 2 classes.


In [7]:
test_data = test_datagen.flow_from_directory(
    VAL_DIR,
    batch_size = 64,
    target_size = (176, 176),
    class_mode = 'binary',
)

Found 256 images belonging to 2 classes.


In [14]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3, 3), input_shape = (176, 176, 3), activation='relu'),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),

                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),

                                    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                      
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(176,176,3)),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    
#     # tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid'),
# ])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 174, 174, 32)      896       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 172, 172, 32)      9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 86, 86, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 84, 84, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 82, 82, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 41, 41, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 39, 39, 64)       

In [15]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [16]:
history = model.fit(
      train_data,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = test_data,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 2.0185 - acc: 0.4883 - val_loss: 0.6905 - val_acc: 0.5000
Epoch 2/15
8/8 [==============================] - 7s 846ms/step - loss: 0.6823 - acc: 0.5664
Epoch 3/15
8/8 [==============================] - 6s 750ms/step - loss: 0.6817 - acc: 0.6208
Epoch 4/15
8/8 [==============================] - 6s 750ms/step - loss: 0.7744 - acc: 0.6341
Epoch 5/15
8/8 [==============================] - 7s 857ms/step - loss: 0.6014 - acc: 0.6914
Epoch 6/15
8/8 [==============================] - 7s 854ms/step - loss: 0.5835 - acc: 0.7266
Epoch 7/15
8/8 [==============================] - 7s 860ms/step - loss: 0.5207 - acc: 0.7305
Epoch 8/15
8/8 [==============================] - 6s 767ms/step - loss: 0.6790 - acc: 0.7051
Epoch 9/15
8/8 [==============================] - 6s 741ms/step - loss: 0.7518 - acc: 0.7472
Epoch 10/15
8/8 [==============================] - 7s 872ms/step - loss: 0.4920 - acc: 0.7539
Epoch 11/15
8/8 [=================